In [1]:
from dynaconf import Dynaconf

# The main structure and the default values
transfer_conf = Dynaconf(
    envvar_prefix = "DYNACONF",
    ignore_unknown_envvars=True, # only previously defined variables
    store_step_enabled=True,
    duplicte_check_enabled=True,
    process_trn_enabled=True,
    dch_checks={
        "extract_sender_dch": True,
        "default_sender_dch": "TESTING",
        "extract_recipient_dch": True,
        "default_recipient_dch": "TESTING",
    },
    name_checks={"check_blocked_list": True},
    rtdr_integrated=True,
    check_step={"check_in_dch": True, "check_out_dch": True},
    finalize_step={
        "insert_reported_file_info_to_db": True,
        "insert_transferred_file_info_to_db": True,
        "update_nrt_transfer_time": True,
        "remove_file_at_end": True,
    },
)


In [40]:
transfer_conf.store_step_enabled

False

In [30]:
import os

# Any environment variable can override
# general
os.environ["DYNACONF_STORE_STEP_ENABLED"] = "false"
os.environ["DYNACONF_DUPLICTE_CHECK_ENABLED"] = "false"
os.environ["DYNACONF_PROCESS_TRN_ENABLED"] = "false"
# dch_checks
os.environ["DYNACONF_dch_checks__EXTRACT_SENDER_DCH"] = "false"
os.environ["DYNACONF_dch_checks__DEFAULT_SENDER_DCH"] = "checkingvalue"
os.environ["DYNACONF_dch_checks__EXTRACT_RECIPIENT_DCH"] = "false"
os.environ["DYNACONF_dch_checks__DEFAULT_RECIPIENT_DCH"] = "helloworld"
# name checks
os.environ["DYNACONF_NAME_CHECKS__CHECK_BLOCKED_LIST"] = "false"
os.environ["DYNACONF_RTDR_INTEGRATED"] = "false"
# check step
os.environ["DYNACONF_CHECK_STEP__CHECK_IN_DCH"] = "false"
os.environ["DYNACONF_CHECK_STEP__CHECK_OUT_DCH"] = "false"

os.environ["DYNACONF_FINALIZE_STEP__INSERT_REPORTED_FILE_INFO_TO_DB"]="false"
os.environ["DYNACONF_FINALIZE_STEP__INSERT_TRANSFERRED_FILE_INFO_TO_DB"]="false"
os.environ["DYNACONF_FINALIZE_STEP__UPDATE_NRT_TRANSFER_TIME"]="false"
os.environ["DYNACONF_FINALIZE_STEP__REMOVE_FILE_AT_END"]="false"
# 


In [2]:
import os

os.environ["DCHROAMKIT_X"] = "1"
os.environ["DCHROAMKIT_Y"] = "varY"
# Re call the constructor

In [49]:
transfer_conf.x

AttributeError: 'Settings' object has no attribute 'X'

In [38]:
from dynaconf import add_converter
add_converter("list", list)
os.environ["DYNACONF_RANDOMVAR"] = "@list test"
# Re call the constructor

In [52]:
#transfer_conf.dch_checks.default_sender_dch
transfer_conf.randomvar

['t', 'e', 's', 't']

In [3]:
from dynaconf import Dynaconf, Validator

transfer_conf.validators.register(
    Validator("token", must_exist=True),
    Validator("test_token", must_exist=True),
)

transfer_conf.validators.validate()

ValidationError: token is required in env main

In [6]:
from enum import Enum, auto
class TYPES(Enum):
    RATE_TYPE = auto(0)
    INTERVAL_TYPE= auto()
    INTERVAL= auto()
    NORMALIZED_TYPE= auto()
    NORMALIZED= auto()
    RATE= auto()
    DURATION_TYPE= auto()
    DURATION= auto()

print(TYPES.RATE_TYPE.value)
print(TYPES.INTERVAL_TYPE.value)
print(TYPES.INTERVAL.value)

0
1
2


In [30]:
text="""
[log_colors]
fore_enable = true
back_enable = false

[log_colors.fore]
TRACE = "LIGHTGREEN_EX"
DEBUG = "LIGHTGREEN_EX"
INFO = "GREEN"
WARNING = "RED"
ERROR = "RED"
FATAL = "WHITE"

# [log_colors.back]
# DEBUG = "BLACK"
# INFO = "BLACK"
# FATAL = "RED"
"""

print(text)


[log_colors]
fore_enable = true
back_enable = false

[log_colors.fore]
TRACE = "LIGHTGREEN_EX"
DEBUG = "LIGHTGREEN_EX"
INFO = "GREEN"
WARNING = "RED"
ERROR = "RED"
FATAL = "WHITE"

# [log_colors.back]
# DEBUG = "BLACK"
# INFO = "BLACK"
# FATAL = "RED"



### Getting values as dictionary

In [31]:
import os
os.environ["ROOT_PATH_FOR_DYNACONF"] = "/tmp"

config_file = "/tmp/config_file.toml"

open(config_file,"w").write(text)


253

In [33]:

from dynaconf import Dynaconf, Validator

conf = Dynaconf(
    settings_files=[os.path.basename(config_file)],
    validators = [
        Validator("log_colors.fore_enable", must_exist= True),
        Validator("log_colors.back_enable", must_exist= True),
        Validator(
            "log_colors.back",
            must_exist=True,
            when=Validator("log_colors.back", must_exist=True, ne=True),
        )
    ]
)

print(conf.log_colors.fore.TRACE)
# print(conf.log_colors.back.get("TRACE"))
# print(conf.log_colors.back.get("FATAL"))
# print(conf.log_colors.back, dict(conf.log_colors.back))

LIGHTGREEN_EX


### Conditional validation

In [149]:
# text="""
# primary_variable = True
# secondary_variable = "HEllo"
# """

# import os
# os.environ["ROOT_PATH_FOR_DYNACONF"] = "/tmp"
# config_file = "/tmp/config_file.toml"
# open(config_file,"w").write(text)

from dynaconf import Dynaconf,Validator

os.environ["DYNACONF_PRODUCTION__SECONDARY_VARIABLE"] = "AMAZING DEFAULT"

config = Dynaconf(
    primary_variable=True,
    #secondary_variable="HEllo world!",
)

config.validators.register(
    # Ensure some parameter meets a condition
    Validator(
        "primary_variable",
        cast=bool,
        must_exist=True,
        #default=[],
    ),
    Validator(
        "secondary_variable",
        must_exist=True,
        when=Validator("primary_variable", eq=False),
        #messages={"must_exist_true": "You forgot to set {name} in your settings."},
        default="TESTING",# Get default if it is must exist, but it is not.
    )
)

config.validators.validate()

print(config.primary_variable)
import os



print(config.get("secondary_variable"))


True
AMAZING DEFAULT


In [140]:

from dynaconf import Dynaconf,Validator

config = Dynaconf()

config.validators.register(
    Validator(
        "var",
        must_exist=True,
        messages={"must_exist_true":"HELLO WORLD"}
    )
)

config.validators.validate()


ValidationError: HELLO WORLD

In [1]:
text="""
[log_colors]
[log_colors.fore]
TRACE = "LIGHTGREEN_EX"
DEBUG = "LIGHTGREEN_EX"
INFO = "GREEN"
WARNING = "RED"
ERROR = "RED"
FATAL = "WHITE"

[log_colors.back]
DEBUG = "BLACK"
INFO = "BLACK"
FATAL = "RED"
"""

import os
os.environ["ROOT_PATH_FOR_DYNACONF"] = "/tmp"
config_file = "/tmp/config_file.toml"
open(config_file,"w").write(text)
from dynaconf import Dynaconf

conf = Dynaconf(
    settings_files=[os.path.basename(config_file)]
)

config.validators.register()

NameError: name 'config' is not defined

In [137]:
from dynaconf import Dynaconf, ValidationError, Validator
from dynaconf import loaders
from dynaconf import settings
from dynaconf.utils.boxing import DynaBox

sorter_conf = Dynaconf(
    settings_files=["anyfile.toml"],
    envvar_prefix="DYNACONF",
    env="production",  # or "production" | "testing"
    production={
        "check_syniverse_sorting_enabled": True,
        # "default_syniverse_sorting_dch": "TESTING",
        "insert_monitor_compressed": True,
        "insert_unknown_reported_file_info_to_db": True,
        "insert_tomiy_reported_file_info_to_db": True,
        "insert_sorted_file": True,
        "extract_recipient_dch": True,
        # "default_recipient_dch": "TESTING",
        "copy_and_remove": True,
    },
    testing={
        "check_syniverse_sorting_enabled": False,
        "default_syniverse_sorting_dch": "TESTING",
        "insert_monitor_compressed": False,
        "insert_unknown_reported_file_info_to_db": False,
        "insert_tomiy_reported_file_info_to_db": False,
        "insert_sorted_file": False,
        "extract_recipient_dch": False,
        "default_recipient_dch": "TESTING",
        "copy_and_remove": False,
    },
)

data = sorter_conf.as_dict(env='production')
loaders.write('/tmp/dynaconf.toml', DynaBox(data).to_dict(), merge=False)

## Getting only the part of the env

In [75]:
from dynaconf import Dynaconf

env = "PRODUCTION"

settings = Dynaconf(
    envvar_prefix="DYNACONF",
    env=env,  # or "production" | "testing"
    production={
        "var": True,
    },
    testing={
        "var": False
    },
)

print(settings[env])

{'var': True}


## Lazy validation

In [2]:
from dynaconf import Dynaconf, Validator
import os

text="""
[production]
var = true

[testing]
var = false

[other]

"""

import os
os.environ["ROOT_PATH_FOR_DYNACONF"] = "/tmp"
config_file = "/tmp/config_file.toml"
open(config_file,"w").write(text)

#env = os.environ.get("ENV_FOR_DYNACONF","production")

settings = Dynaconf(
    settings_files=[os.path.basename(config_file)],
    #envvar_prefix="DYNACONF",
    environments=True,
    #env=None,  # or "production" | "testing"
    #force_env=None,# Forcing
    default_env="testing",
    new_var7= "var7 contetn"
    #validate_only_current_env=False,
    #var= True,
    # production={
    #     "var": True,
    # },
    # testing={
    #     "var": False
    # },
    # validators=[
    #     Validator(
    #     "var",
    #     must_exist=True
    # )
    #]
)

print(settings.to_dict())
print(settings.get("var"))
# print(settings.production.to_dict())
# print(settings.production.var)

# settings.validators.register(
#     Validator(
#         "var",
#         must_exist=True
#     )
# )

# print(settings.to_dict())

#settings.validators.validate()


{'NEW_VAR7': 'var7 contetn', 'VAR': False}
False


In [12]:
sorter_conf = Dynaconf(
    settings_files=["config_file.toml"],
    #envvar_prefix="DYNACONF",
    environments=True,
    #env="testing",  # or "production" | "testing"
    #force_env=None,# Forcing
    default_env="production",
    #PRODUCTION= {'var': True}, TESTING= {'var': False}, OTHER= {},
    #var="ASDASD"
)
print(sorter_conf.to_dict())
print(sorter_conf.get("var"))
#print(sorter_conf.get("check_syniverse_sorting_enabled"))

{'VAR': True}
True


In [6]:
from dynaconf import Dynaconf, Validator
import re

# Custom validator function to check for prevent SQL injection
def __validate_db_name(value) -> bool:
    """
    >>> __validate_db_name("test")
    True
    >>> __validate_db_name("test12")
    True
    >>> __validate_db_name("sql -- injection")
    False
    >>> __validate_db_name("test.test")
    False
    >>> __validate_db_name("test.test")
    False
    """
    return bool(re.match("^[^0-9][a-zA-Z0-9_]{2,62}$", value))


# Add the custom validator to your Dynaconf configuration
db_config = Dynaconf(
    settings_files=["db_config.toml"],
    envvar_prefix="DCHROAMKIT",
    environments=True,
    default_env="testing",
    validate_only_current_env=True,
    # Validators
    validators=[
        Validator("mysql_main_database", must_exist=True, condition=__validate_db_name),
        Validator("mysql_tapdb_database", must_exist=True, condition=__validate_db_name),
    ],
)

print(db_config.mysql_main_database)


ValidationError: mysql_main_database invalid for __validate_db_name(anyname -- test) in env DEVELOPMENT